In [1]:
!pip install cvxpy tqdm --quiet
!pip install ecos --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 5.4 MB/s eta 0:00:00


### Giới thiệu vấn đề
Vấn đề cân bằng xe đạp là một bài toán kinh điển trong học tăng cường (RL), nơi mục tiêu là giữ cho xe đạp cân bằng bằng cách thực hiện các hành động thích hợp (ví dụ: mô-men xoắn lái) để duy trì sự ổn định. Mã Python cung cấp ba phương pháp để giải quyết vấn đề này: Quy hoạch động (DP), Quy hoạch tuyến tính (LP) và Quy hoạch tuyến tính xấp xỉ (ALP). Bài thuyết trình này giới thiệu mã, làm nổi bật các thành phần chính và so sánh hiệu suất của các phương pháp này.

Code sẽ phân tách không gian trạng thái, mô hình hóa động lực học của xe đạp, tính toán ma trận chuyển tiếp và phần thưởng, và áp dụng ba phương pháp RL để đưa ra chính sách tối ưu. Dưới đây, chúng ta sẽ đi qua thiết lập vấn đề, các đoạn mã chính và kết quả đánh giá.

### Thiết lập vấn đề
Vấn đề cân bằng xe đạp liên quan đến việc điều khiển một chiếc xe đạp để giữ thẳng đứng bằng cách điều chỉnh góc lái. Trạng thái của hệ thống được xác định bởi bốn biến:

phi: Góc của xe đạp so với phương thẳng đứng (góc nghiêng).
phi_dot: Tốc độ góc của nghiêng.
delta: Góc lái.
delta_dot: Tốc độ góc của góc lái.
Không gian hành động bao gồm ba hành động rời rạc: áp dụng mô-men xoắn âm (trái), không có mô-men xoắn (giữa), hoặc mô-men xoắn dương (phải). Mục tiêu là tối đa hóa phần thưởng tích lũy trong khi ngăn xe đạp ngã (tức là giữ phi trong giới hạn).

#### Import thư viện

In [2]:
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from collections import defaultdict
import time, tracemalloc


#### Rời rạc hóa trạng thái
Để làm cho vấn đề có thể giải quyết được, không gian trạng thái liên tục được phân tách thành một lưới 12 bin cho mỗi chiều, dẫn đến 20.736 trạng thái (12×12×12×12). Mã định nghĩa các khoảng và phân tách các biến trạng thái như sau:

In [3]:
# Rời rạc hóa mỗi biến thành 9 mức
n_bins = 12
phi_range = (-0.5, 0.5)
phi_dot_range = (-1.0, 1.0)
delta_range = (-0.3, 0.3)
delta_dot_range = (-1.0, 1.0)

phi_vals = np.linspace(*phi_range, n_bins)
phi_dot_vals = np.linspace(*phi_dot_range, n_bins)
delta_vals = np.linspace(*delta_range, n_bins)
delta_dot_vals = np.linspace(*delta_dot_range, n_bins)

all_states = list(product(range(n_bins), repeat=4))
state_id = {s: i for i, s in enumerate(all_states)}
num_states = len(all_states)
actions = [-1, 0, 1]  # trái, giữa, phải
num_actions = len(actions)

print(f"Số trạng thái rời rạc: {num_states}, số hành động: {num_actions}")

Số trạng thái rời rạc: 20736, số hành động: 3


## Các thành phần mã chính
#### Động lực học xe đạp
Hàm bicycle_next_state mô hình hóa động lực học của xe đạp dựa trên các tham số vật lý (trọng lực, chiều dài, lực ma sát) và mô-men xoắn áp dụng. Nó tính toán trạng thái tiếp theo bằng cách sử dụng tích phân Euler

In [4]:
def bicycle_next_state(state, action, dt=0.1):
    phi, phi_dot, delta, delta_dot = state

    g = 9.8
    l = 1.0
    b = 0.1
    max_delta = 0.3

    torque = 0.02 * action
    phi_ddot = (g / l) * np.sin(phi) + delta_dot
    delta_ddot = torque - b * delta_dot - phi

    phi_dot += phi_ddot * dt
    phi += phi_dot * dt
    delta_dot += delta_ddot * dt
    delta += delta_dot * dt
    delta = np.clip(delta, -max_delta, max_delta)

    return np.array([phi, phi_dot, delta, delta_dot])


Hàm discretize_state ánh xạ các trạng thái liên tục trở lại lưới rời rạc:

In [5]:
def discretize_state(state):
    phi, phi_dot, delta, delta_dot = state
    phi_idx = np.digitize(phi, phi_vals) - 1
    phi_dot_idx = np.digitize(phi_dot, phi_dot_vals) - 1
    delta_idx = np.digitize(delta, delta_vals) - 1
    delta_dot_idx = np.digitize(delta_dot, delta_dot_vals) - 1
    return (
        np.clip(phi_idx, 0, n_bins - 1),
        np.clip(phi_dot_idx, 0, n_bins - 1),
        np.clip(delta_idx, 0, n_bins - 1),
        np.clip(delta_dot_idx, 0, n_bins - 1),
    )

def undiscretize_index(idx):
    return np.array([
        phi_vals[idx[0]],
        phi_dot_vals[idx[1]],
        delta_vals[idx[2]],
        delta_dot_vals[idx[3]]
    ])


#### Ma trận xác suất chuyển tiếp và phần thưởng
Mã xây dựng một ma trận chuyển tiếp $T$ và ma trận phần thưởng $R$. Đối với mỗi cặp trạng thái-hành động, nó tính toán trạng thái tiếp theo và gán một phần thưởng (mặc định: -1, hoặc -100 nếu xe đạp ngã, tức là $|\phi|$ > 0.5):


In [6]:
T = dict()
R = np.full((num_states, num_actions), -1.0)

for s_idx, s_tuple in enumerate(all_states):
    T[s_idx] = dict()
    for a_idx, a in enumerate(actions):
        s_continuous = undiscretize_index(s_tuple)
        s_next_continuous = bicycle_next_state(s_continuous, a)
        s_next_tuple = discretize_state(s_next_continuous)

        if abs(s_next_continuous[0]) > 0.5:
            R[s_idx, a_idx] = -100.0
            s_next_tuple = s_tuple

        s_next_idx = state_id.get(s_next_tuple, s_idx)
        T[s_idx][a_idx] = [(s_next_idx, 1.0)]

print("✅ Đã tính xong T và R.")


✅ Đã tính xong T và R.


### Thuật toán Quy hoạch động (DP):

DP giải quyết MDP bằng cách tính toán lặp đi lặp lại value function $V(s)$ cho từng trạng thái sử dụng phương trình Bellman.

Trong code được cung cấp, Giá trị lặp cập nhật hàm giá trị $V(s)$ cho tất cả các trạng thái cho đến khi hội tụ, sau đó rút ra chính sách tối ưu $\pi(s)$ bằng cách chọn hành động tối đa hóa phần thưởng kỳ vọng cộng với giá trị tương lai đã chiết khấu.  
Các bước chính:  
+ Khởi tạo $V(s) = 0$ cho tất cả các trạng thái.  
+ Cập nhật $V(s) \leftarrow \max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V(s') \right]$ cho đến khi sự thay đổi trong $V(s)$ dưới một ngưỡng.  
+ Trích xuất chính sách tham lam: $\pi(s) = \arg\max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V(s') \right]$.  

In [7]:
gamma = 0.95
threshold = 1e-3
V_dp = np.zeros(num_states)
iteration = 0

# Bắt đầu đo thời gian và bộ nhớ
start = time.time()
tracemalloc.start()

while True:
    delta = 0
    V_new = np.zeros_like(V_dp)
    for s in range(num_states):
        V_new[s] = max(
            R[s, a] + gamma * sum(p * V_dp[s2] for s2, p in T[s][a])
            for a in range(num_actions)
        )
        delta = max(delta, abs(V_new[s] - V_dp[s]))
    V_dp = V_new
    iteration += 1
    if delta < threshold:
        break

# Tính chính sách greedy từ giá trị V_dp
pi_dp = np.zeros(num_states, dtype=int)
for s in range(num_states):
    pi_dp[s] = int(np.argmax([
        R[s, a] + gamma * sum(p * V_dp[s2] for s2, p in T[s][a])
        for a in range(num_actions)
    ]))

# Dừng đo bộ nhớ và thời gian
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
elapsed = time.time() - start

print(f"✅ Value Iteration hội tụ sau {iteration} vòng.")
print(f"⏱️ Thời gian: {elapsed:.2f}s | 💾 Peak memory: {peak / 1e6:.2f}MB")


✅ Value Iteration hội tụ sau 226 vòng.
⏱️ Thời gian: 137.70s | 💾 Peak memory: 0.36MB


### Quy hoạch tuyến tính (LP):
LP tái cấu trúc MDP thành một bài toán tối ưu hóa tuyến tính, tối thiểu hóa tổng các giá trị trạng thái $\sum_s V(s)$ dưới các ràng buộc Bellman.

Các ràng buộc đảm bảo rằng hàm giá trị thỏa mãn $V(s) \geq R(s,a) + \gamma \sum_{s'} P(s'|s,a) V(s')$ cho tất cả các trạng thái $s$ và hành động $a$.
Giải pháp cung cấp hàm giá trị tối ưu $V(s)$, từ đó chính sách được rút ra một cách tham lam.  
Các bước chính:
+ Cấu trúc LP với các biến $V(s)$ cho mỗi trạng thái.  
+ Thêm các ràng buộc cho mỗi cặp trạng thái-hành động.  
Giải quyết bằng cách sử dụng một bộ giải LP (ví dụ: scipy.optimize.linprog với phương pháp "highs").  
+ Tính toán chính sách bằng cách tối đa hóa hàm giá trị hành động.

In [8]:
from scipy.optimize import linprog
import time, tracemalloc

# ⏱️ Bắt đầu đo hiệu suất
start = time.time()
tracemalloc.start()

# ⚙️ Xây dựng ràng buộc LP: v_s >= R(s,a) + γ * ∑ P(s'|s,a) * v_s'
A_ub = []
b_ub = []

for s in range(num_states):
    for a in range(num_actions):
        if not T[s][a]:
            continue  # Bỏ qua nếu hành động không hợp lệ
        row = np.zeros(num_states)
        row[s] = -1  # v_s chuyển sang vế trái
        for s2, p in T[s][a]:  # T[s][a] là list of (s', p)
            row[s2] += gamma * p
        A_ub.append(row)
        b_ub.append(-R[s, a])  # RHS chuyển dấu sang phải

# 🎯 Hàm mục tiêu: minimize ∑ v_s
c = np.ones(num_states)

# 🧮 Giải bài toán LP
res_lp = linprog(
    c,
    A_ub=A_ub,
    b_ub=b_ub,
    method="highs",
    options={"tol": 1e-9}
)

# 📈 Lấy giá trị trạng thái từ nghiệm tối ưu
V_lp = res_lp.x

# 🧭 Suy ra chính sách greedy từ giá trị LP
pi_lp = np.zeros(num_states, dtype=int)
for s in range(num_states):
    q_values = [
        R[s, a] + gamma * sum(p * V_lp[s2] for s2, p in T[s][a])
        if T[s][a] else -np.inf
        for a in range(num_actions)
    ]
    pi_lp[s] = int(np.argmax(q_values))

# 🧮 Dừng đo hiệu suất
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
elapsed = time.time() - start

print("✅ Đã giải xong LP (phiên bản tối ưu hóa).")
print(f"⏱️ Thời gian: {elapsed:.2f}s | 💾 Peak memory: {peak / 1e6:.2f}MB")

/tmp/ipykernel_19/1363212621.py:27: OptimizeWarning: Unrecognized options detected: {'tol': 1e-09}. These will be passed to HiGHS verbatim.
  res_lp = linprog(
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linprog_highs.py:355: OptimizeWarning: Unrecognized options detected: {'tol': 1e-09}
  res = _highs_wrapper(c, A.indptr, A.indices, A.data, lhs, rhs,


✅ Đã giải xong LP (phiên bản tối ưu hóa).
⏱️ Thời gian: 37.58s | 💾 Peak memory: 30974.79MB


### Thuật toán Quy hoạch tuyến tính xấp xỉ (ALP):


#### Chọn đặc trưng
Hàm đặc trưng phi(s) cho bài toán Bicycle Balancing  
Trạng thái s là index, ánh xạ đến tuple (phi, phi_dot, delta, delta_dot)

In [9]:
def get_features(s):
    # Giải mã lại trạng thái từ chỉ số s
    phi_idx, phi_dot_idx, delta_idx, delta_dot_idx = all_states[s]

    # Chuẩn hóa mỗi thành phần về [0, 1]
    phi_norm = phi_idx / (n_bins - 1)
    phi_dot_norm = phi_dot_idx / (n_bins - 1)
    delta_norm = delta_idx / (n_bins - 1)
    delta_dot_norm = delta_dot_idx / (n_bins - 1)

    # Trả về vector đặc trưng phi(s)
    return np.array([
        1.0,
        phi_norm,
        phi_dot_norm,
        delta_norm,
        delta_dot_norm,
        phi_norm ** 2,
        phi_dot_norm ** 2,
        delta_norm ** 2,
        delta_dot_norm ** 2,
        phi_norm * delta_norm,
        phi_dot_norm * delta_dot_norm,
        phi_norm * delta_dot_norm,
        phi_dot_norm * delta_norm,
    ])

Phi = np.array([get_features(s) for s in range(num_states)])


ALP giải quyết vấn đề khả năng mở rộng của LP bằng cách xấp xỉ hàm giá trị dưới dạng một tổ hợp tuyến tính của các đặc trưng: $V(s) \approx \phi(s)^T \theta$, trong đó $\phi(s)$ là một vector đặc trưng cho trạng thái $s$, và $\theta$ là một vector tham số.  
Thay vì tối ưu hóa trên tất cả các trạng thái, ALP tối ưu hóa trên không gian đặc trưng, giảm số lượng biến từ $|S|$ xuống kích thước đặc trưng $|\phi|$.  
LP tối thiểu hóa $\sum |\theta|$ (hoặc một mục tiêu liên quan) dưới các ràng buộc đảm bảo rằng hàm giá trị xấp xỉ thỏa mãn bất đẳng thức Bellman.  
Các bước chính:  
+ Định nghĩa một ma trận đặc trưng $\Phi$ trong đó mỗi hàng là $\phi(s)$.  
+ Cấu trúc các ràng buộc: $\phi(s)^T \theta \geq R(s,a) + \gamma \sum_{s'} P(s'|s,a) \phi(s')^T \theta$.  
+ Giải cho $\theta$, tính toán $V(s) = \phi(s)^T \theta$, và rút ra chính sách một cách tham lam.  

In [10]:
from scipy.optimize import linprog
from scipy.sparse import lil_matrix
import time, tracemalloc

# Bắt đầu đo hiệu suất
start = time.time()
tracemalloc.start()

phi_dim = Phi.shape[1]  # Số chiều đặc trưng

# 🔧 Xây dựng ràng buộc dưới dạng A_ub @ theta <= b_ub
num_constraints = sum(len(T[s][a]) > 0 for s in range(num_states) for a in range(num_actions))
A_ub = lil_matrix((num_constraints, phi_dim))  # sparse matrix
b_ub = []

row_idx = 0
for s in range(num_states):
    phi_s = Phi[s]
    for a in range(num_actions):
        if not T[s][a]: continue  # skip nếu hành động không hợp lệ
        expected_phi = np.zeros(phi_dim)
        for s2, p in T[s][a]:
            expected_phi += p * Phi[s2]
        A_ub[row_idx, :] = expected_phi * gamma - phi_s  # chuyển vế
        b_ub.append(-R[s, a])  # RHS
        row_idx += 1

# 🎯 Hàm mục tiêu: minimize ∑|theta| ≈ minimize ∑ theta (với ràng buộc phù hợp)
# Ở đây ta minimize ∑ theta để tương ứng với chuẩn L1 (theta >= 0 sẽ đảm bảo tương đương)
c = np.ones(phi_dim)

# 🧮 Giải bài toán LP với solver "highs"
res = linprog(
    c,
    A_ub=A_ub,
    b_ub=b_ub,
    method="highs"
)

# Kiểm tra kết quả
if not res.success:
    raise ValueError("Không giải được bài toán ALP bằng linprog:", res.message)

theta_opt = res.x
V_alp = Phi @ theta_opt

# 🧭 Suy ra chính sách greedy từ V_alp
pi_alp = np.zeros(num_states, dtype=int)
for s in range(num_states):
    q_vals = []
    for a in range(num_actions):
        expected = sum(p * V_alp[s2] for s2, p in T[s][a])
        q_vals.append(R[s, a] + gamma * expected)
    pi_alp[s] = int(np.argmax(q_vals))

# 🧮 Dừng đo hiệu suất
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
elapsed = time.time() - start

print("✅ Đã giải xong ALP (phiên bản chuẩn LP dùng linprog).")
print(f"⏱️ Thời gian: {elapsed:.2f}s | 💾 Peak memory: {peak / 1e6:.2f}MB")

✅ Đã giải xong ALP (phiên bản chuẩn LP dùng linprog).
⏱️ Thời gian: 28.09s | 💾 Peak memory: 77.83MB


In [11]:
from scipy.optimize import linprog
from scipy.sparse import lil_matrix
import time, tracemalloc

# === Sampling-Based ALP ===

# ⚙️ Sampling trạng thái theo occupancy từ chính sách ban đầu (VD: random)
sampled_states = []
num_samples = 7500
np.random.seed(42)

for _ in range(num_samples):
    s = np.random.choice(num_states)
    a = np.random.choice(num_actions)
    if T[s][a]:  # chỉ chọn (s, a) hợp lệ
        sampled_states.append((s, a))

# Bắt đầu đo hiệu suất
start = time.time()
tracemalloc.start()
c
phi_dim = Phi.shape[1]
A_ub = lil_matrix((len(sampled_states), phi_dim))
b_ub = []

for idx, (s, a) in enumerate(sampled_states):
    phi_s = Phi[s]
    expected_phi = np.zeros(phi_dim)
    for s2, p in T[s][a]:
        expected_phi += p * Phi[s2]
    A_ub[idx, :] = expected_phi * gamma - phi_s
    b_ub.append(-R[s, a])

# 🎯 Hàm mục tiêu: minimize ∑ θ
c = np.ones(phi_dim)

# 🧮 Giải ALP (sampling version)
res = linprog(
    c,
    A_ub=A_ub,
    b_ub=b_ub,
    method="highs"
)

if not res.success:
    raise ValueError("Không giải được ALP (sampling):", res.message)

theta_opt_sampled = res.x
V_alp_sampled = Phi @ theta_opt_sampled

# 🧭 Suy ra chính sách greedy từ ALP sampling
pi_alp_sampled = np.zeros(num_states, dtype=int)
for s in range(num_states):
    q_vals = []
    for a in range(num_actions):
        expected = sum(p * V_alp_sampled[s2] for s2, p in T[s][a])
        q_vals.append(R[s, a] + gamma * expected)
    pi_alp_sampled[s] = int(np.argmax(q_vals))

# Hiệu suất
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
elapsed = time.time() - start

print("✅ Đã giải xong ALP (sampling).")
print(f"⏱️ Thời gian: {elapsed:.2f}s | 💾 Peak memory: {peak / 1e6:.2f}MB")


✅ Đã giải xong ALP (sampling).
⏱️ Thời gian: 4.00s | 💾 Peak memory: 9.40MB


In [12]:
from scipy.optimize import linprog
from scipy.sparse import lil_matrix
import time, tracemalloc

# === ALP with Prioritized Sampling ===

# ⚙️ Bước 1: Tính Bellman error từ V gần đúng (VD: dùng V_dp)
errors = []
pairs = []
for s in range(num_states):
    for a in range(num_actions):
        if not T[s][a]: continue
        expected = sum(p * V_dp[s2] for s2, p in T[s][a])
        error = abs(V_dp[s] - (R[s, a] + gamma * expected))
        errors.append(error)
        pairs.append((s, a))

# ⚖️ Bước 2: Chuẩn hóa lỗi thành phân phối xác suất
errors = np.array(errors)
probs = errors + 1e-6  # tránh 0
probs /= probs.sum()

# 🎲 Bước 3: Sampling theo Bellman error
num_samples = 7500
np.random.seed(42)
sampled_indices = np.random.choice(len(pairs), size=num_samples, replace=False, p=probs)
sampled_states = [pairs[i] for i in sampled_indices]

# 🚀 Bắt đầu giải ALP
start = time.time()
tracemalloc.start()

phi_dim = Phi.shape[1]
A_ub = lil_matrix((num_samples, phi_dim))
b_ub = []

for idx, (s, a) in enumerate(sampled_states):
    phi_s = Phi[s]
    expected_phi = np.zeros(phi_dim)
    for s2, p in T[s][a]:
        expected_phi += p * Phi[s2]
    A_ub[idx, :] = expected_phi * gamma - phi_s
    b_ub.append(-R[s, a])

# 🎯 Hàm mục tiêu: minimize ∑ θ
c = np.ones(phi_dim)

# 🧮 Giải ALP với prioritized sampling
res = linprog(
    c,
    A_ub=A_ub,
    b_ub=b_ub,
    method="highs"
)

if not res.success:
    raise ValueError("Không giải được ALP (prioritized):", res.message)

theta_opt_prioritized = res.x
V_alp_prioritized = Phi @ theta_opt_prioritized

# 🧭 Suy ra chính sách greedy từ ALP prioritized
pi_alp_prioritized = np.zeros(num_states, dtype=int)
for s in range(num_states):
    q_vals = []
    for a in range(num_actions):
        expected = sum(p * V_alp_prioritized[s2] for s2, p in T[s][a])
        q_vals.append(R[s, a] + gamma * expected)
    pi_alp_prioritized[s] = int(np.argmax(q_vals))

# ⏱️ Hiệu suất
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
elapsed = time.time() - start

print("✅ Đã giải xong ALP (prioritized sampling).")
print(f"⏱️ Thời gian: {elapsed:.2f}s | 💾 Peak memory: {peak / 1e6:.2f}MB")


✅ Đã giải xong ALP (prioritized sampling).
⏱️ Thời gian: 3.79s | 💾 Peak memory: 9.40MB


### Kết quả:

In [13]:
# 🎯 Hàm đánh giá reward trung bình của một chính sách
def evaluate_policy(pi, episodes=1000):
    rewards = []
    for _ in range(episodes):
        s = np.random.choice(num_states)
        total_reward = 0
        for _ in range(100):
            a = pi[s]
            if not T[s][a]:  # tránh lỗi khi hành động không hợp lệ
                break
            next_s, prob = T[s][a][0]
            total_reward += R[s, a]
            s = next_s
        rewards.append(total_reward)
    return np.mean(rewards)

# 🎯 Reward trung bình
r_dp  = evaluate_policy(pi_dp)
r_lp  = evaluate_policy(pi_lp)
r_alp = evaluate_policy(pi_alp)
r_alp_sampled = evaluate_policy(pi_alp_sampled)
r_alp_prioritized_sampled = evaluate_policy(pi_alp_prioritized)

# 📏 Sai số giữa V_DP và các V khác
max_err_lp  = np.max(np.abs(V_dp - V_lp))
mean_err_lp = np.mean(np.abs(V_dp - V_lp))

max_err_alp  = np.max(np.abs(V_dp - V_alp))
mean_err_alp = np.mean(np.abs(V_dp - V_alp))

max_err_sampled  = np.max(np.abs(V_dp - V_alp_sampled))
mean_err_sampled = np.mean(np.abs(V_dp - V_alp_sampled))

# 🧭 So sánh chính sách
policy_diff_lp  = np.sum(pi_dp != pi_lp)
policy_diff_alp = np.sum(pi_dp != pi_alp)
policy_diff_sampled = np.sum(pi_dp != pi_alp_sampled)
policy_diff_prioritized_sampled = np.sum(pi_dp!= pi_alp_prioritized)

# 📊 In kết quả
print("🎯 Reward trung bình mỗi chính sách:")
print(f"  - DP   : {r_dp:.4f}")
print(f"  - LP   : {r_lp:.4f}")
print(f"  - ALP  : {r_alp:.4f}")
print(f"  - ALP (sampling): {r_alp_sampled:.4f}")
print(f"  - ALP (prioritized sampling): {r_alp_prioritized_sampled:.4f}\n")

print("📏 Sai số giá trị trạng thái so với DP:")
print(f"  - LP   : max = {max_err_lp:.4f}, mean = {mean_err_lp:.4f}")
print(f"  - ALP  : max = {max_err_alp:.4f}, mean = {mean_err_alp:.4f}")
print(f"  - ALP (sampling): max = {max_err_sampled:.4f}, mean = {mean_err_sampled:.4f}\n")

print("🧭 Số trạng thái có chính sách khác với DP:")
print(f"  - LP   : {policy_diff_lp} / {num_states}")
print(f"  - ALP  : {policy_diff_alp} / {num_states}")
print(f"  - ALP (sampling): {policy_diff_sampled} / {num_states}")


🎯 Reward trung bình mỗi chính sách:
  - DP   : -8773.2910
  - LP   : -9167.7070
  - ALP  : -9158.8960
  - ALP (sampling): -9171.3700
  - ALP (prioritized sampling): -9078.9040

📏 Sai số giá trị trạng thái so với DP:
  - LP   : max = 1999.9815, mean = 1565.1108
  - ALP  : max = 1999.9815, mean = 1565.1108
  - ALP (sampling): max = 1999.9815, mean = 1565.1108

🧭 Số trạng thái có chính sách khác với DP:
  - LP   : 204 / 20736
  - ALP  : 204 / 20736
  - ALP (sampling): 204 / 20736
